In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout, GRU, Bidirectional
from sklearn.model_selection import train_test_split


In [2]:

# 1. 범용적인 split_sequence 함수 정의 (다양한 시계열 데이터 적용 가능)
def split_sequence(sequence, n_steps):
    """
    시계열 데이터를 n_step 길이로 분할하는 함수.
    - sequence: 전체 데이터 시퀀스 (리스트나 배열 형태)
    - n_steps: 시퀀스를 나누는 기준이 되는 step 수
   
    출력: 입력 시퀀스 X, 출력 시퀀스 y
    """
    X, y = list(), list()
    for i in range(len(sequence)):
        # i번째 데이터부터 i+n_steps까지 하나의 샘플로 만듬
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


In [3]:

# 2. 예시 데이터셋 생성 (랜덤 시계열 데이터)
np.random.seed(42)
data = np.random.rand(1000)  # 1000개의 랜덤 시계열 데이터 생성
n_steps = 3  # 시계열 데이터를 n=3 스텝씩 나눔

# 시계열 데이터를 split_sequence 함수로 나눔
X, y = split_sequence(data, n_steps)

# 3. CNN+LSTM 모델을 위한 입력 데이터의 형상 조정
# CNN과 LSTM 레이어는 3D 데이터를 입력으로 받기 때문에, 데이터 차원을 맞춰야 함
# (샘플 수, 타임스텝 수, 특징 수)의 형상으로 변환해야 함
X = X.reshape((X.shape[0], X.shape[1], 1))  # 특징 수는 1 (단일 시계열 특징이므로)

# 4. 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
import warnings
warnings.filterwarnings('ignore')

# 5. CNN + LSTM 모델 정의
model = Sequential()

# (1) Conv1D 층: CNN을 사용하여 시계열 데이터의 지역적 패턴을 학습
# - Conv1D 대신에 Conv2D를 사용하면 이미지나 다차원 데이터에 적합한 모델로 변형 가능
# - filters: CNN 필터 개수, kernel_size: 필터 크기
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, 1)))

# (2) MaxPooling1D 층: MaxPooling을 사용해 데이터 크기를 줄임
# - GlobalMaxPooling1D를 사용하면 전체 시계열 데이터에서 최대값을 추출 가능
model.add(MaxPooling1D(pool_size=2))

# (3) Dropout 층: Dropout을 사용해 과적합을 방지 (0.2 = 20% 노드 비활성화)
# - Dropout 대신에 BatchNormalization을 사용할 수 있음 (네트워크 안정화)
model.add(Dropout(0.2))  # Dropout을 통해 과적합 방지

# (4) LSTM 층: CNN에서 추출한 특징을 기반으로 시계열 정보를 처리
# - LSTM 대신에 GRU를 사용할 수 있음 (GRU는 LSTM보다 경량화된 RNN 구조)
# - Bidirectional(LSTM(50))을 사용하면 양방향 LSTM 적용 가능
model.add(LSTM(50, activation='relu', return_sequences=False))  # return_sequences=False로 마지막 출력만 사용

# (5) 완전 연결층(Dense Layer)
# - Dense 층은 CNN+RNN에서 추출한 특징을 종합해 예측을 수행
# - Dense 대신에 다른 구조를 사용할 수 있음 (예: Self-attention layer)
model.add(Dense(50, activation='relu'))

# (6) 출력층 (회귀 문제를 가정하므로 활성화 함수는 없거나 선형 사용)
# - 분류 문제인 경우 softmax 또는 sigmoid를 사용할 수 있음
model.add(Dense(1))  # 회귀 문제에서는 단일 값을 출력

# 6. 모델 컴파일: 회귀 문제이므로 MSE 손실 함수 사용
# - 분류 문제에서는 loss='categorical_crossentropy' 또는 'binary_crossentropy' 사용 가능
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 2, 64)          │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        23,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,793 (100.75 KB)

 Trainable params: 25,793 (100.75 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:

# 7. 모델 학습
model.fit(X_train, y_train, epochs=50, verbose=1, validation_data=(X_test, y_test))


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.2746 - mae: 0.4398 - val_loss: 0.1240 - val_mae: 0.2888
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1014 - mae: 0.2671 - val_loss: 0.0917 - val_mae: 0.2664
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0890 - mae: 0.2537 - val_loss: 0.0907 - val_mae: 0.2656
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0877 - mae: 0.2556 - val_loss: 0.0894 - val_mae: 0.2632
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0917 - mae: 0.2604 - val_loss: 0.0891 - val_mae: 0.2636
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0909 - mae: 0.2603 - val_loss: 0.0885 - val_mae: 0.2624
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0807 - mae: 0.2425 - val_loss: 0.0884 - val_mae: 0.2623
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0824 - mae: 0.2456 - val_loss: 0.0884 - val_mae: 0.2619
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0875 - mae:

In [7]:

# 8. 평가
mse, mae = model.evaluate(X_test, y_test, verbose=1)
print(f"Test MSE: {mse}, Test MAE: {mae}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0970 - mae: 0.2794 
Test MSE: 0.0885838195681572, Test MAE: 0.2614727318286896
